In [25]:
from utils import getAllImageInfo
import pandas as pd

labels_clean = pd.read_csv('labels_clean.csv')
print(labels_clean.shape)

(6000, 2)


In [26]:
from PIL import Image
from os import path

rawImgDir = './images/faces'
rawImgExtention = '.png'
aspectRatioTotal = 0
rawImgFullPath = []
# 宽高比不对的图片（横向拍照）
badAspectRatioImgCnt = 0

for index, row in labels_clean.iterrows():
    user_id = row['user_id']
    fullPath = path.join(rawImgDir, str(user_id) + rawImgExtention)
    im = Image.open(fullPath)
    aspectRatio = im.size[0] / im.size[1]
    if aspectRatio >= 1:
        badAspectRatioImgCnt += 1
        labels_clean.at[index, 'user_id'] = -1
    else:
        aspectRatioTotal += aspectRatio
        rawImgFullPath.append(fullPath)

averageAspectRatio = aspectRatioTotal / (len(labels_clean['user_id']) - badAspectRatioImgCnt)
labels_clean.drop(labels_clean[labels_clean['user_id'] <= 0].index, inplace=True)
labels_clean.to_csv('labels_final.csv', index=False)

print('bad image count:', badAspectRatioImgCnt)
print('average acpect ratio is', averageAspectRatio)
print(labels_clean.shape)

bad image count: 4
average acpect ratio is 0.5919884089392928
(5996, 2)


In [24]:
import math

resizedImgDir = './images/resized'
resizedImgFileExtension = '.jpg'
minWidth, minHeight = float('inf'), float('inf')
cropedSizes = []

for fullPath in rawImgFullPath:
    im = Image.open(fullPath)
    w, h = im.size[0], im.size[1]
    
    croppedSize = ()
    croppedWidth = 0
    croppedHeight = 0
    
    if w / h > averageAspectRatio:
        cut = w - h * averageAspectRatio
        cutL = math.floor(cut / 2)
        cutR = math.floor(cut -cutL)
        if cutL == 0 & cutR == 0: continue
        croppedSize = (cutL, 0, w - cutR, h)
        croppedWidth = w - cutL - cutR
        croppedHeight = h
    else:
        cut = h - w / averageAspectRatio
        cutT = math.floor(cut / 2)
        cutB = math.floor(cut -cutT)
        if cutT == 0 & cutB == 0: continue
        croppedSize = (0, cutT, w, h - cutB)
        croppedWidth = w
        croppedHeight = h - cutT - cutB

    cropedSizes.append(croppedSize)
    minWidth = croppedWidth if minWidth > croppedWidth else minWidth
    minHeight = croppedHeight if minHeight > croppedHeight else minHeight

print('minWidth:', minWidth, 'minHeight:', minHeight)

for fullPath, croppedSize, user_id in zip(rawImgFullPath, cropedSizes, labels_clean['user_id']):
    im = Image.open(fullPath)
    w, h = im.size[0], im.size[1]
    resizedImgFullPath = path.join(resizedImgDir, str(user_id) + resizedImgFileExtension)

    resizedIm = im.crop(croppedSize).resize((minWidth, minHeight), Image.BILINEAR)
    resizedIm.save(resizedImgFullPath)
    

minWidth: 379 minHeight: 640
